### Wallet pairs exploration

In [22]:
# Read datasets and merge
df = pd.read_csv('data/data_features.csv')
df1 = pd.read_csv('data/data_sybils_pairs.csv')
df1['target'] = 1

df = df.merge(df1, how = 'left')
df['target'] = df['target'].fillna(0).astype(int)

print(df.shape)
df[:3]

(37612, 11)


,round_name,wallets,tenure_year,frequency_year,monetary_year,consistency_year,tenure_round,frequency_round,monetary_round,consistency_round,target
0,1_gitcoin_gr15,0x0000ce08fa224696a819877070bf378e8b131acf_0xa...,297.27,2.00,0.21,1.00,NaN,NaN,NaN,NaN,0
1,1_gitcoin_gr15,0x0003a2d21b35c7cfc0fb259c9e27dbdb434864bd_0x3...,31.70,1.00,0.01,1.00,NaN,NaN,NaN,NaN,0
2,1_gitcoin_gr15,0x0003a2d21b35c7cfc0fb259c9e27dbdb434864bd_0x4...,274.64,1.00,0.04,1.00,NaN,NaN,NaN,NaN,0


In [23]:
len_wal = lambda x: len(set('_'.join(x).split('_')))

dict_sybils = df1.groupby('round_name')['wallets'].agg(len_wal).to_dict()
dict_sybils

{'1_gitcoin_gr15': 729,
 '2_unicef': 157,
 '3_fantom': 251,
 '41_gitcoin_climate': 142,
 '42_gitcoin_ethereum': 167,
 '43_gitcoin_oss': 169}

### 1. Statistics

In [24]:
# Basic stat
df2 = pd.read_csv('data/data_rounds.csv')
df2 = df2.groupby('round_name').agg({
    'source_wallet': lambda x: len(set(x)),
    # 'destination_wallet': lambda x: len(set(x)),
    # 'amt_usd': [len, sum],    
})
df2.columns = [
    'Number of donors',
    # 'Number of projects',
    # 'Number of votes',
    # 'Donated sum'
]
# df1['Average number of votes per donor'] = df1['Number of votes']/df1['Number of donors']
# df1['Average sum per donor'] = df1['Donated sum']/df1['Number of donors']
df2.reset_index(inplace = True)

In [25]:
# Adding stat
f1 = df.tenure_round.notnull()
f2 = df.target == 1

dict1 = df.groupby('round_name')['wallets'].agg(len_wal).to_dict()
df2['Connected donors per year'] = df2.round_name.map(dict1)

dict1 = df[f1].groupby('round_name')['wallets'].agg(len_wal).to_dict()
df2['Connected donors per round'] = df2.round_name.map(dict1)

df2['Sybils by hash algorithm'] = df2.round_name.map(dict_sybils)

dict1 = df[f2].groupby('round_name')['wallets'].agg(len_wal).to_dict()
df2['Sybils approved by trnx per year'] = df2.round_name.map(dict1)

dict1 = df[f1&f2].groupby('round_name')['wallets'].agg(len_wal).to_dict()
df2['Sybils approved by trnx per round'] = df2.round_name.map(dict1)

df2

,round_name,Number of donors,Connected donors per year,Connected donors per round,Sybils by hash algorithm,Sybils approved by trnx per year,Sybils approved by trnx per round
0,1_gitcoin_gr15,58409,18154,5722,729,194,99
1,2_unicef,15510,4490,1429,157,12,2
2,3_fantom,16344,4259,1370,251,124,43
3,41_gitcoin_climate,3399,274,99,142,41,27
4,42_gitcoin_ethereum,2901,578,130,167,14,6
5,43_gitcoin_oss,11706,2744,637,169,62,33


### 2. Generate output file with potential Sybil wallets

In [35]:
# Hash algo
df1 = pd.read_csv('data/data_sybils.csv')
df1 = df1.groupby(['feat_round','source_wallet']).size().reset_index()
del df1[0]
df1.columns = ['round_name','wallet']
df1['flg_hash_algo'] = 1

In [36]:
# Trnx verification
wal = lambda x: set('_'.join(x).split('_'))
f1 = df.tenure_round.notnull()
f2 = df.target == 1

###
list1 = list()
list2 = list()
x = df[f2].groupby('round_name')['wallets'].agg(wal)
for ind,x1 in x.iteritems():
    list1+=list(x1)
    list2+=[ind]*len(x1)
df0 = pd.DataFrame({
    'round_name': list2,
    'wallet': list1
})
feat = 'flg_trnx_year'
df0[feat] = 1
df1 = df1.merge(df0, how = 'left')
df1[feat] = df1[feat].fillna(0).astype(int)

###
list1 = list()
list2 = list()
x = df[f2&f1].groupby('round_name')['wallets'].agg(wal)
for ind,x1 in x.iteritems():
    list1+=list(x1)
    list2+=[ind]*len(x1)
df0 = pd.DataFrame({
    'round_name': list2,
    'wallet': list1
})
feat = 'flg_trnx_round'
df0[feat] = 1
df1 = df1.merge(df0, how = 'left')
df1[feat] = df1[feat].fillna(0).astype(int)

df1.to_csv('data/wallets_sybils.csv', index = False)
print(df1.shape)
df1[:5]

(1615, 5)


,round_name,wallet,flg_hash_algo,flg_trnx_year,flg_trnx_round
0,1_gitcoin_gr15,0x00a41e72a3bf61c444a021b63a0d5f5af0dc1b83,1,0,0
1,1_gitcoin_gr15,0x00f7b390192724d35b48286406fd6cbe650e0255,1,0,0
2,1_gitcoin_gr15,0x0104d476c19c6c3f9b5536f6aaf62ac4bee9188b,1,1,0
3,1_gitcoin_gr15,0x0212535d92b4fabc447e88e0676cfe3b233730bc,1,1,1
4,1_gitcoin_gr15,0x0356c28c048b52b568e35edd834e463ae2ef723b,1,0,0


In [38]:
df1.groupby('round_name').sum()

,flg_hash_algo,flg_trnx_year,flg_trnx_round
round_name,,,
1_gitcoin_gr15,729,194,99
2_unicef,157,12,2
3_fantom,251,124,43
41_gitcoin_climate,142,41,27
42_gitcoin_ethereum,167,14,6
43_gitcoin_oss,169,62,33
